In [ ]:
import numpy as np
import scipy as sp
import scipy.linalg as linalg
import torch
import matplotlib.pyplot as plt
import imageio
from qutip import (fock, coherent, basis, destroy, qeye, tensor, wigner, mesolve, ptrace)

import pickle
im

## Task A

In [ ]:
# depickle the data located at '../data/synthetic/... pickle files
# and save them as .npy files

data = pickle.load(open('../data/synthetic/noisy_wigner_0.pickle', 'rb'))

